In [2]:
import json
import pandas as pd
import jieba
from jieba import posseg
import numpy as np
from loader import *
import torch
import sys
from modules import embedding, encoder, model_sbj
from sklearn.model_selection import KFold

In [8]:
data = []
file_path = '../data/train_data.json'
with open(file_path, 'r') as f:
    for line in f:
        tmp = json.loads(line)
        data.append(tmp)

In [9]:
data[0]

{'postag': [{'word': '如何', 'pos': 'r'},
  {'word': '演', 'pos': 'v'},
  {'word': '好', 'pos': 'a'},
  {'word': '自己', 'pos': 'r'},
  {'word': '的', 'pos': 'u'},
  {'word': '角色', 'pos': 'n'},
  {'word': '，', 'pos': 'w'},
  {'word': '请', 'pos': 'v'},
  {'word': '读', 'pos': 'v'},
  {'word': '《', 'pos': 'w'},
  {'word': '演员自我修养', 'pos': 'nw'},
  {'word': '》', 'pos': 'w'},
  {'word': '《', 'pos': 'w'},
  {'word': '喜剧之王', 'pos': 'nw'},
  {'word': '》', 'pos': 'w'},
  {'word': '周星驰', 'pos': 'nr'},
  {'word': '崛起', 'pos': 'v'},
  {'word': '于', 'pos': 'p'},
  {'word': '穷困潦倒', 'pos': 'a'},
  {'word': '之中', 'pos': 'f'},
  {'word': '的', 'pos': 'u'},
  {'word': '独门', 'pos': 'n'},
  {'word': '秘笈', 'pos': 'n'}],
 'text': '如何演好自己的角色，请读《演员自我修养》《喜剧之王》周星驰崛起于穷困潦倒之中的独门秘笈',
 'spo_list': [{'predicate': '主演',
   'object_type': '人物',
   'subject_type': '影视作品',
   'object': '周星驰',
   'subject': '喜剧之王'}]}

In [29]:
data_1 = []
file_path = '../data/dev_data.json'
with open(file_path, 'r') as f:
    for line in f:
        tmp = json.loads(line)
        data_1.append(line)

In [9]:
data_np = np.array(data)

In [23]:
splits = KFold(n_splits=10, shuffle=True, random_state=333).split(list(range(len(data_np))))

In [24]:
for train_index, val_index in splits:
    tmp = data_np[val_index]
    break

In [25]:
len(tmp)

19475